In [0]:
!pip install --upgrade tables
!pip install eli5
!ppip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 
/bin/bash: ppip: command not found


In [0]:
import pandas as pd
import numpy as np 

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
cd 'drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car'


/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [0]:
ls

data/                     day3_simple_model.ipynb   LICENSE
day2_visualization.ipynb  dw_matrix_car_day1.ipynb  README.md


In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
 SUFFIX_CAT = '__cat'
 for feat in df.columns:

   if  isinstance(df[feat][0], list): continue
   factorized_values = df[feat].factorize()[0]
   if SUFFIX_CAT in feat:
     df[feat] = factorized_values

   else:
    df[feat + SUFFIX_CAT] = factorized_values
   

In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
 
  scores = cross_val_score(model, X, y, cv=3, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

##DecisionTree

In [0]:
run_model( DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

## Random Forest

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model( model, cat_feats)

(-18718.657185256638, 64.5424578125788)

##XGBoost

In [0]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed':0
}

run_model( xgb.XGBRegressor(**xgb_params), cat_feats)

[21:11:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:11:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:11:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [0]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[20:34:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [0]:
len(cat_feats)

151

In [0]:
feats = [
         'param_napęd__cat',
         'param_rok-produkcji__cat',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc__cat',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa__cat',
         'seller_name__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat'
]

In [0]:
len(feats)

20

In [0]:
run_model( xgb.XGBRegressor(**xgb_params), feats)

[21:18:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:18:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:18:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [0]:
df['param_napęd'].unique()

array([None, 'Na przednie koła', '4x4 (dołączany automatycznie)',
       'Na tylne koła', '4x4 (dołączany ręcznie)', '4x4 (stały)'],
      dtype=object)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))


In [0]:
feats = [
         'param_napęd__cat',
         'param_rok-produkcji',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc__cat',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa__cat',
         'seller_name__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat'
]
run_model( xgb.XGBRegressor(**xgb_params), feats)

[21:18:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:19:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:19:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]) )

In [0]:
df['param_moc'].unique()

array([ 90, 115, 262, 110, 310, 105, 140, 175, 125, 185, 190, 440, 141,
       200, 224,  75,  99, 184, 109, 233, 116,  68, 286, 126, 160, 135,
       120, 272,  -1, 150, 180, 136, 102, 131, 218, 245, 170, 112, 250,
       252,  73, 100, 313, 101, 285,  70, 383, 174, 277, 132, 130, 215,
        60, 330, 163, 177,  98,  78, 189, 156, 143,  69, 113,  65, 122,
        82, 251,  95, 197, 235, 238, 171, 381, 400, 178,  80, 165,  85,
       258, 142, 204, 124,  55, 144, 231, 248, 152, 181, 210, 340, 129,
       147,  50,  54, 290, 306, 193,  77, 164,  96, 194, 111, 166, 206,
       118, 360, 211, 271, 455, 280, 106, 114, 421,  74, 213, 121, 275,
       435, 384, 326,  88, 220, 260,  64,  86, 128, 256, 240, 244, 162,
       237, 350,  35, 265, 202, 133,  83, 117, 146,  92, 192, 145, 525,
       254, 182, 328, 367, 148, 456,  97, 270, 107, 108, 203, 155,  94,
        93, 241,  20,  71, 173,  58, 205, 236,   1, 557,  84, 457,  72,
       295, 134, 425, 228,  81, 230, 201,  87, 234, 299, 585, 20

In [0]:
feats = [
         'param_napęd__cat',
         'param_rok-produkcji',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa__cat',
         'seller_name__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat'
]
run_model( xgb.XGBRegressor(**xgb_params), feats)

[21:19:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:19:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:19:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.450230340148, 62.2164408321879)

In [0]:
df['param_pojemność-skokowa'].unique()

array([   898,   1560,   3000, ...,   5992,   1966, 142280])

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')))

In [0]:
feats = [
         'param_napęd__cat',
         'param_rok-produkcji',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa',
         'seller_name__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat'
]
run_model( xgb.XGBRegressor(**xgb_params), feats)

[21:19:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:19:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:19:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)